In [1]:
import sys
import os
import time
import random
import math
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import pandas as pd
from torch.nn.utils.rnn import pad_sequence

In [3]:
from models import Gen

In [4]:
# Parameters
word_emb_dim = 100  # size of word embeddings
label_emb_dim = 100  # size of label embeddings
hid_dim = 100  # number of hidden units
nlayers = 1  # number of lstm layers
nclass = 4  # number of classes
dropout = 0
use_cuda = torch.cuda.is_available()
tied = False
use_bias = False
concat_label = 'hidden'
avg_loss = False
one_hot = False
bit_width=8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
data_dict = torch.load(os.path.join('../data', 'ag_news', 'data', 'traindata.v40000.l80.s5000'))
traindata = data_dict['traindata']
trainlabel = data_dict['trainlabel']
validdata = data_dict['validdata']
validlabel = data_dict['validlabel']
testdata = data_dict['testdata']
testlabel = data_dict['testlabel']
vocab_size = data_dict['vocabsize']

In [7]:
model = Gen(vocab_size, word_emb_dim, label_emb_dim, hid_dim, nlayers, nclass, dropout, use_cuda, tied, use_bias, concat_label, avg_loss, one_hot, bit_width).to(device)

In [8]:
model_path = '../ModelParameterLSTM_FP.pth'

# Load the state dictionary from the file
state_dict = torch.load(model_path)

# Load the state dictionary into the model
model.load_state_dict(state_dict)

<All keys matched successfully>

In [14]:
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision

from brevitas.export import export_onnx_qcdq
from brevitas.export import export_torch_qcdq
from brevitas.graph.equalize import activation_equalization_mode
from brevitas.graph.quantize import preprocess_for_quantize
from brevitas.graph.target.flexml import preprocess_for_flexml_quantize
from brevitas_examples.imagenet_classification.ptq.ptq_common import apply_act_equalization
from brevitas_examples.imagenet_classification.ptq.ptq_common import apply_bias_correction
from brevitas_examples.imagenet_classification.ptq.ptq_common import apply_gpfq
from brevitas_examples.imagenet_classification.ptq.ptq_common import apply_gptq
from brevitas_examples.imagenet_classification.ptq.ptq_common import apply_learned_round_learning
from brevitas_examples.imagenet_classification.ptq.ptq_common import calibrate
from brevitas_examples.imagenet_classification.ptq.ptq_common import calibrate_bn
from brevitas_examples.imagenet_classification.ptq.ptq_common import quantize_model
from brevitas_examples.imagenet_classification.ptq.utils import add_bool_arg
from brevitas_examples.imagenet_classification.ptq.utils import get_model_config
from brevitas_examples.imagenet_classification.ptq.utils import get_torchvision_model
from brevitas_examples.imagenet_classification.utils import generate_dataloader
from brevitas_examples.imagenet_classification.utils import SEED
from brevitas_examples.imagenet_classification.utils import validate

In [15]:
model_config = get_model_config(model)

In [17]:
quant_model = quantize_model(
    model,
    backend='fx',  # or 'layerwise' based on your preference
    weight_bit_width=8,
    act_bit_width=8,
    bias_bit_width=32,
    weight_quant_granularity='per_tensor',
    act_quant_percentile=99.999,
    act_quant_type='sym',
    scale_factor_type='float_scale',
    quant_format='int',
    device='cpu'  # or 'cuda' if using GPU
)

AttributeError: 'Gen' object has no attribute 'graph'